In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from transformers import pipeline
from wordcloud import WordCloud
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

/home/digo_eu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-13 17:51:39.204712: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-13 17:51:40.069010: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[]


2023-04-13 17:51:41.120481: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-13 17:51:41.120959: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


<h3>Detecção de encoding</h3>
Leitura deste dataset retornou erro sem definição do encoding.
<pre><code>
import chardet
with open('DisneylandReviews.csv', 'rb') as rawdata:
    result = chardet.detect(rawdata.read())
result
</code></pre>
Esta detecção levou vários minutos, mas foi necessária porque caracteres encontrados em seções avançadas do documento indicaram encoding 'latin-1' ou 'ISO-8859-1'.<br>
Fonte do dataset indicou, no entanto, o uso de encoding 'cp1252'.

In [3]:
df = pd.read_csv('DisneylandReviews.csv', encoding='cp1252')
print(df.shape)
print(df.nunique())
df.head()

(42656, 6)
Review_ID            42636
Rating                   5
Year_Month             112
Reviewer_Location      162
Review_Text          42632
Branch                   3
dtype: int64


,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


In [4]:
df.drop_duplicates(subset='Review_Text', inplace=True, keep='first')

In [5]:
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def modelo_lexical(sentence):
    sentiment = sia.polarity_scores(sentence)
    print(sentiment)
    
model_neural = pipeline('sentiment-analysis', max_length=1024)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/digo_eu/nltk_data...
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [6]:
modelo_lexical("I don't feel very well")
modelo_lexical("I feel stoked about this!")

print(model_neural("I don't feel very well"))
print(model_neural("I feel stoked about this!"))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


{'neg': 0.404, 'neu': 0.596, 'pos': 0.0, 'compound': -0.2572}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
[{'label': 'NEGATIVE', 'score': 0.9997064471244812}]
[{'label': 'POSITIVE', 'score': 0.9998165965080261}]


In [7]:
resid = {}

for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['Review_Text']
        myid = row['Review_ID'] 
        resid[myid] = model_neural(text)
    except RuntimeError:
        print(f'Falha em id {myid}')

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
import csv

with open('resultado_rede_neural.csv', mode='w', newline='') as f:
    writer = csv.writer(f)
    for key, value in resid.items():
        writer.writerow([key, value])

In [ ]:
resultado = pd.read_csv('resultado.csv', sep=';')

resultado